In [2]:
from functools import partial, reduce
from itertools import accumulate, repeat, chain, takewhile, tee, islice
import more_itertools
import operator

import numpy as np
import sympy as sym
import random

In [3]:
def take(n):
    return lambda iterable: islice(iterable, 0, n)

def head(iterable, n=10):
    return list(take(n)(iterable))

# def last(iterable):
#     try:
#         while True:
#             x = next(iterable)
#     except StopIteration:
#         return x
    
last = more_itertools.last
        
def pipeline(*func_list):
    def compose_two(f, g):
        return lambda x: g(f(x))
    return reduce(compose_two, func_list)

def pipeline_eval(x, *func_list):
    def apply(x, f):
        return f(x)
    return reduce(apply, func_list, x)

# def filter_printer(iterable):
#     for element in iterable:
#         print(element)
#         yield element
        
def print_and_return(x):
    print(x)
    return x

filter_printer = partial(map, print_and_return)

In [4]:
class Markov:
    def __init__(self, state_space, P):
        self.state_space = state_space
        if callable(P):
            self.next_state = P
        else:
            if isinstance(P, sym.Matrix):
                P = P.tolist()
            try:
                self.weights = [[float(i) for i in row] for row in P]
                self.index_dict = {state:index for index,state in enumerate(state_space)}
                self.next_state = lambda state: random.choices(state_space, 
                                weights=self.weights[self.index_dict[state]])[0]
            except:
                raise ValueError("Illegal argument for probability transition matrix: \n{}".format(P))

    def chain(self, state0):
        state = state0
        while True:
            yield state
            state = self.next_state(state)

In [5]:
state_space = [1,2,3,4,5,6]
P = sym.Matrix([
        [0, 6, 0, 0, 0, 0],
        [3, 0, 0, 3, 0, 0],
        [0, 0, 0, 0, 6, 0],
        [0, 3, 0, 0, 3, 0],
        [0, 0, 2, 2, 0, 2],
        [0, 0, 0, 0, 6, 0]])/6

markov = Markov(state_space, P)

pipeline_eval(markov.chain(4),
              take(10),
              list)

[4, 2, 1, 2, 1, 2, 4, 2, 4, 2]

In [20]:
def until(condition, seq):
    for element in seq:
        yield element
        if condition(element):
            break

def seq_len(seq):
    return sum(1 for _ in seq)

def sub1(x):
    return x - 1

def trial():
    return pipeline_eval(
        markov.chain(4),
        partial(until, lambda state: state in [1, 6]),
        seq_len,
        sub1)

# a
N = 10**5
average = sum(trial() for _ in range(N))/N
print(average)

4.99006


In [10]:
# b
def count_if(state):
    return state == 6

def trial():
    return pipeline_eval(
        markov.chain(4),
        partial(until, lambda state: state in [1]),
        partial(map, count_if),
        sum)

N = 10**5
average = sum(trial() for _ in range(N))/N
print(average)

2.00233


In [11]:
# c
def found_food(final_state):
    return final_state == 1

def trial():
    return pipeline_eval(
        markov.chain(4),
        partial(until, lambda state: state in [1, 6]),
        last,
        found_food)

N = 10**5
prob = sum(trial() for _ in range(N))/N
print(prob)

0.50147
